## NB: python3-only

In [2]:
from zipfile import ZipFile
from io import BytesIO
import urllib.request

ZipFile.extract(
    ZipFile(
        BytesIO(
            urllib
            .request
            .urlopen('http://s3.amazonaws.com/alexa-static/top-1m.csv.zip')
            .read()
        )
    ),
    'top-1m.csv'
)

'/home/tgorlenko/Spark/HW/top-1m.csv'

# Дз №1. Мапредьюс руками

Напишите маппер и редьюсер для задачи обратного индекса по рейтингу сайтов алексы.
Задача построения обратного индекса означает, что нужно разбить все домены в рейтинге на части(токены) по символу точки, а на выходе получить пары (токен, все сайты в названии которых содержался токен). Пример:

```
(
    google.com,
    google.ru,
    ya.ru,
    hse.ru
)

```
должно на выходе превратиться в:

```python
(
    (com, (google.com,)),
    (google, (google.com, google.ru)),
    (ru, (google.ru, ya.ru, hse.ru)),
    (ya, (ya.ru, )),
    (hse, (hse.ru, ))
)
```


In [1]:
with open('top-1m.csv') as alexa:
    alexa_domains = [s.strip().split(",")[1] for s in alexa]

## 1.1 Маппер

In [2]:
def mapper(domain):
    """
    Маппер для обратного индекса. Разбивает доменное имя на токены и помечает каждый токен доменом. 
    Пример: google.com -> ('com', 'google.com'), ('google', 'google.com')
    """
    # Ваш код здесь
    result = tuple(map(lambda x: (x, domain), domain.split('.')))
    return result
    

In [3]:
mapper('google.com')

(('google', 'google.com'), ('com', 'google.com'))

In [4]:
mapper('com')

(('com', 'com'),)

## 1.2 Редьюсер

In [5]:
def reducer(token_domain_pairs):
    """
    Редьюсер для обратного индекса. По набору кортежей, созданных маппером, возвращает кортеж 
    вида (токен, (домен1, домен2, ...)
    Пример:
    (('google', 'google.com'), ('google', 'google.se')) -> ('google', ('google.com', 'google.se'))
    Примечание: гарантируется, что у всех кортежей на входе оданковый токен.
    """
    # Ваш код здесь
    
    key = token_domain_pairs[0][0]
    domains = tuple(map(lambda x: x[1], token_domain_pairs))
    
    return (key, domains)
    

    

In [6]:
reducer((('google', 'google.com'), ('google', 'google.se')) )

('google', ('google.com', 'google.se'))

## bonus:

Напишите также несколько тестов, чтобы проверить корректность маппера и редьюсера

например таких:

In [7]:
assert set(mapper("ya.ru")) == {("ya", "ya.ru"), ("ru", "ya.ru")}
assert set(mapper("com")) == {("com", "com")}

def reducer_test(testcase, expected_result):
    actual_result = reducer(testcase)
    assert actual_result[0] == expected_result[0]
    assert set(actual_result[1]) == set(expected_result[1])
    
reducer_test(
    (('google', 'google.com'), ('google', 'google.se')),
    ('google', ('google.com', 'google.se'))
)

In [8]:
def flatmap(mapper_func, iterable):
    from itertools import chain
    return tuple(chain.from_iterable(map(mapper_func, iterable)))

#### проверим как работает ваш маппер на датасете:

In [9]:
flatmap(mapper, alexa_domains[:10])

(('google', 'google.com'),
 ('com', 'google.com'),
 ('youtube', 'youtube.com'),
 ('com', 'youtube.com'),
 ('tmall', 'tmall.com'),
 ('com', 'tmall.com'),
 ('baidu', 'baidu.com'),
 ('com', 'baidu.com'),
 ('qq', 'qq.com'),
 ('com', 'qq.com'),
 ('taobao', 'taobao.com'),
 ('com', 'taobao.com'),
 ('sohu', 'sohu.com'),
 ('com', 'sohu.com'),
 ('facebook', 'facebook.com'),
 ('com', 'facebook.com'),
 ('login', 'login.tmall.com'),
 ('tmall', 'login.tmall.com'),
 ('com', 'login.tmall.com'),
 ('yahoo', 'yahoo.com'),
 ('com', 'yahoo.com'))

## 1.3 Шаффл
напишите функцию, которая сгруппирует всю выдачу всех мапперов по токену, так чтобы соблюдалось предположение, принятое нами в определении редьюсера:

> `Примечание: гарантируется, что у всех кортежей на входе оданковый токен.`

#### Примечание: в идеале, в реализации шафла не пользоваться питоньими диктами (`{k:v}` и т.д )

In [10]:
dataset = (
    ('google', 'google.com'),
    ('com', 'google.com'),
    ('tmall', 'tmall.com'),
    ('com', 'tmall.com'),
    ('sohu', 'sohu.com'),
    ('com', 'sohu.com'),
    ('google', 'google.tmall'),
    ('tmall', 'google.tmall')
          )

In [11]:
def shuffle(dataset):
    """
    Группирует с выводом мапперов по ключу. Пример:
    
    (
        ('google', 'google.com'),
        ('com', 'google.com'),
        ('youtube', 'youtube.com'),
        ('com', 'youtube.com')
    )

    --> shuffle все превращает в --> 

    (
        [('com', 'google.com'), ('com', 'youtube.com')],
        [('google', 'google.com')],
        [('youtube', 'youtube.com')],
    )

    :param dataset:
    :return:
    
    """
    # Ваш код здесь
    sorted_data = sorted(dataset)
    result = []
    current_key = sorted_data[0][0]
    pairs_list = []
    for pair in sorted_data:
        if current_key == pair[0]:
            pairs_list.append(pair)
        else:
            result.append(pairs_list)
            current_key = pair[0]
            pairs_list = [pair]
        if pair == sorted_data[-1]:
            result.append(pairs_list)

    return tuple(result) 

In [12]:
shuffle(dataset)

([('com', 'google.com'), ('com', 'sohu.com'), ('com', 'tmall.com')],
 [('google', 'google.com'), ('google', 'google.tmall')],
 [('sohu', 'sohu.com')],
 [('tmall', 'google.tmall'), ('tmall', 'tmall.com')])

### Собираем все вместе

In [13]:
mapper_output = flatmap(mapper, alexa_domains)
shuffled = shuffle(mapper_output)
reduced = map(reducer, shuffled)

reversed_idx = dict(reduced)

Должно посчитаться секунд за пять. Если не получается и считается долго, то прежде чем оптимизировать код попробуйте подставить небольшое подмножество `alexa_domains`, например `alexa_domains[:100]`. Корректность кода важнее быстродействия.

`reversed_idx` должен содержать то что нужно для поиска по токену:

In [14]:
reversed_idx['yandex']

('auto.yandex',
 'chef.yandex',
 'clickhouse.yandex',
 'driver.yandex',
 'eda.yandex',
 'ir.yandex',
 'metrika.yandex',
 'yandex.by',
 'yandex.com',
 'yandex.com.ge',
 'yandex.com.tr',
 'yandex.ee',
 'yandex.fr',
 'yandex.github.io',
 'yandex.kz',
 'yandex.lv',
 'yandex.md',
 'yandex.net',
 'yandex.org.kz',
 'yandex.ru',
 'yandex.tm',
 'yandex.ua',
 'yandex.uz')

In [10]:
reversed_idx['yandex']

('yandex.ee',
 'yandex.uk.com',
 'yandex.kz',
 'yandex.kg',
 'yandex.net',
 'yandex.tm',
 'yandex.lv',
 'yandex.lt',
 'clickhouse.yandex',
 'yandex.ru',
 'yandex.uz',
 'yandex.az',
 'yandex.tj',
 'driver.yandex',
 'auto.yandex',
 'yandex.com.ge',
 'yandex.com',
 'yandex.co.il',
 'yandex.com.tr',
 'eda.yandex',
 'yandex.fr',
 'yandex.md',
 'ir.yandex',
 'yandex.org.kz',
 'yandex.by',
 'yandex.ua')